# 配方设计计算器

## 功能说明

本脚本根据历史酿造数据的提取系数/效率统计值，计算达到目标麦汁所需的原料用量和工艺参数。

### 计算步骤

1. **计算麦芽重量**: 根据目标麦汁体积、比重、熬煮后效率和理论浸出率
2. **计算糖化用水量**: 根据麦芽重量和水料比
3. **估计糖化后比重**: 根据洗糟前提取系数，用于过程监控
4. **计算洗糟水用量**: 根据谷物吸收率和熬煮蒸发量
5. **估计洗糟后比重**: 根据洗糟后提取系数，用于过程监控

### 计算公式

$$M_{\text{act}}(V, SG) = 2.5 \times (SG - 1) \times V \times SG$$

$$\text{提取系数} = \frac{M_{\text{Actual Extract}}}{M_{\text{Potential Extract}}}$$

$$M_{\text{Potential Extract}} = \text{麦芽重量} \times \text{理论浸出率}$$

$$\text{Grain Absorption Rate} = \frac{\text{糖化水} + \text{洗糟水} - \text{熬煮前体积}}{\text{麦芽重量}}$$

**注意**：
- 提取系数为小数形式（如0.7078表示70.78%）
- 洗糟后提取系数使用总水量计算，可能大于1

In [39]:
import pandas as pd
import numpy as np
from pathlib import Path

print("📊 配方设计计算器")
print("="*60)

📊 配方设计计算器


## 1. 加载历史效率数据

In [40]:
# 从CSV文件加载历史提取系数/效率数据
metrics_file = 'brewing_metrics_summary.csv'
metrics_df = pd.read_csv(metrics_file)

# 提取关键指标（注意：这些值现在是小数形式，如0.7078表示70.78%）
pre_sparge_coefficient = metrics_df['pre_sparge_coefficient'].values[0]
post_sparge_coefficient = metrics_df['post_sparge_coefficient'].values[0]
grain_absorption_rate = metrics_df['grain_absorption_rate'].values[0]
post_boil_efficiency = metrics_df['post_boil_efficiency'].values[0]
boil_off_volume = metrics_df['boil_off_volume'].values[0]

print("✅ 历史提取系数/效率数据加载成功！")
print("-"*60)
print(f"洗糟前提取系数: {pre_sparge_coefficient:.4f} ({pre_sparge_coefficient*100:.2f}%)")
print(f"洗糟后提取系数: {post_sparge_coefficient:.4f} ({post_sparge_coefficient*100:.2f}%)")
print(f"谷物吸收率: {grain_absorption_rate:.3f} L/kg")
print(f"熬煮后效率: {post_boil_efficiency:.4f} ({post_boil_efficiency*100:.2f}%)")
print(f"熬煮蒸发量: {boil_off_volume:.2f} L")
print("="*60)

✅ 历史提取系数/效率数据加载成功！
------------------------------------------------------------
洗糟前提取系数: 0.7077 (70.77%)
洗糟后提取系数: 0.7947 (79.47%)
谷物吸收率: 0.862 L/kg
熬煮后效率: 0.6869 (68.69%)
熬煮蒸发量: 1.70 L


## 2. 设置酿造目标

In [41]:
# ==================== 酿造目标参数 ====================
# 目标麦汁参数
target_post_boil_volume = 22.0  # 煮沸后麦汁体积 (L)
target_post_boil_sg = 1.060     # 煮沸后麦汁比重

# 工艺参数
potential_extract_rate = 0.78   # 理论浸出率 (综合值)
water_grain_ratio = 3.2         # 水料比

print("🎯 酿造目标参数")
print("-"*60)
print(f"煮沸后麦汁体积: {target_post_boil_volume} L")
print(f"煮沸后麦汁比重: {target_post_boil_sg}")
print(f"理论浸出率: {potential_extract_rate*100}%")
print(f"水料比: {water_grain_ratio}")
print("="*60)

🎯 酿造目标参数
------------------------------------------------------------
煮沸后麦汁体积: 22.0 L
煮沸后麦汁比重: 1.06
理论浸出率: 78.0%
水料比: 3.2


## 3. 定义计算函数

In [42]:
def calculate_actual_extract(volume_l, sg):
    """
    计算麦汁中的实际含糖量
    
    参数:
        volume_l: 麦汁体积 (L)
        sg: 麦汁比重
    
    返回:
        实际含糖量 (kg)
    """
    return 2.5 * (sg - 1.0) * volume_l * sg

def calculate_malt_weight(target_volume, target_sg, efficiency, potential_rate):
    """
    计算所需麦芽重量
    
    参数:
        target_volume: 目标麦汁体积 (L)
        target_sg: 目标麦汁比重
        efficiency: 效率 (%)
        potential_rate: 理论浸出率
    
    返回:
        麦芽重量 (kg)
    """
    actual_extract = calculate_actual_extract(target_volume, target_sg)
    potential_extract = actual_extract / (efficiency / 100)
    malt_weight = potential_extract / potential_rate
    return malt_weight

def estimate_sg_from_sugar(volume_l, sugar_mass):
    """
    根据糖量估计比重（基于糖量守恒）
    
    参数:
        volume_l: 麦汁体积 (L)
        sugar_mass: 糖的质量 (kg)
    
    返回:
        估计的比重
    
    说明：
        从 sugar_mass = 2.5 * (sg - 1) * volume * sg 反解sg
        2.5 * volume * sg^2 - 2.5 * volume * sg - sugar_mass = 0
    """
    a = 2.5 * volume_l
    b = -2.5 * volume_l
    c = -sugar_mass
    
    # 求解二次方程: a*sg^2 + b*sg + c = 0
    discriminant = b**2 - 4*a*c
    sg = (-b + discriminant**0.5) / (2*a)
    
    return sg

print("✅ 计算函数定义完成！")

✅ 计算函数定义完成！


## 4. 配方计算

### 步骤1: 计算麦芽重量

In [43]:
# 计算所需麦芽重量
total_malt_weight = calculate_malt_weight(
    target_post_boil_volume,
    target_post_boil_sg,
    post_boil_efficiency * 100,  # 转换为百分比形式
    potential_extract_rate
)

print("📋 步骤1: 计算麦芽重量")
print("-"*60)
print(f"目标麦汁实际含糖量: {calculate_actual_extract(target_post_boil_volume, target_post_boil_sg):.3f} kg")
print(f"使用熬煮后效率: {post_boil_efficiency:.4f} ({post_boil_efficiency*100:.2f}%)")
print(f"所需麦芽重量: {total_malt_weight:.3f} kg")
print("="*60)

📋 步骤1: 计算麦芽重量
------------------------------------------------------------
目标麦汁实际含糖量: 3.498 kg
使用熬煮后效率: 0.6869 (68.69%)
所需麦芽重量: 6.528 kg


### 步骤2: 计算糖化用水量

In [44]:
# 计算糖化用水量
strike_water = total_malt_weight * water_grain_ratio

print("📋 步骤2: 计算糖化用水量")
print("-"*60)
print(f"麦芽重量: {total_malt_weight:.3f} kg")
print(f"水料比: {water_grain_ratio}")
print(f"糖化用水量: {strike_water:.2f} L")
print("="*60)

📋 步骤2: 计算糖化用水量
------------------------------------------------------------
麦芽重量: 6.528 kg
水料比: 3.2
糖化用水量: 20.89 L


### 步骤3: 估计糖化后比重（用于过程监控）

In [45]:
# 估计糖化后比重
# 首先计算糖化后的实际糖量
potential_extract_total = total_malt_weight * potential_extract_rate
sugar_mash = potential_extract_total * pre_sparge_coefficient

# 假设糖化后的麦汁体积约等于糖化用水量（麦芽吸收部分水后的体积）
mash_volume = strike_water  # 简化假设

estimated_pre_sparge_sg = estimate_sg_from_sugar(mash_volume, sugar_mash)

print("📋 步骤3: 估计糖化后比重")
print("-"*60)
print(f"麦芽理论总糖量: {potential_extract_total:.3f} kg")
print(f"糖化后实际糖量: {sugar_mash:.3f} kg (使用洗糟前提取系数 {pre_sparge_coefficient:.4f})")
print(f"糖化后估计体积: {mash_volume:.2f} L")
print(f"估计糖化后比重: {estimated_pre_sparge_sg:.4f}")
print(f"  (过程监控目标值)")
print("="*60)

📋 步骤3: 估计糖化后比重
------------------------------------------------------------
麦芽理论总糖量: 5.092 kg
糖化后实际糖量: 3.604 kg (使用洗糟前提取系数 0.7077)
糖化后估计体积: 20.89 L
估计糖化后比重: 1.0648
  (过程监控目标值)


### 步骤4: 计算洗糟水用量

In [46]:
# 计算熬煮前体积
pre_boil_volume = target_post_boil_volume + boil_off_volume

# 根据谷物吸收率计算洗糟水用量
# grain_absorption_rate = (strike_water + sparge_water - pre_boil_volume) / total_malt_weight
# sparge_water = grain_absorption_rate * total_malt_weight + pre_boil_volume - strike_water
sparge_water = grain_absorption_rate * total_malt_weight + pre_boil_volume - strike_water

print("📋 步骤4: 计算洗糟水用量")
print("-"*60)
print(f"熬煮蒸发量: {boil_off_volume:.2f} L")
print(f"熬煮前体积: {pre_boil_volume:.2f} L")
print(f"谷物吸收率: {grain_absorption_rate:.3f} L/kg")
print(f"洗糟水用量: {sparge_water:.2f} L")
print("="*60)

📋 步骤4: 计算洗糟水用量
------------------------------------------------------------
熬煮蒸发量: 1.70 L
熬煮前体积: 23.70 L
谷物吸收率: 0.862 L/kg
洗糟水用量: 8.43 L


### 步骤5: 估计洗糟后比重（用于过程监控）

In [47]:
# 估计洗糟后比重（用于过程监控）
# 
# 【重要】洗糟后提取系数的定义：
# η_洗糟后 = M_act(总水量, SG_洗糟后) / M_potential
# 其中总水量 = strike_water + sparge_water
#
# 因此，监控目标比重也应该基于总水量计算，与提取系数的定义保持一致

# 已知量
potential_extract_total = total_malt_weight * potential_extract_rate  # 理论总糖量

# 使用熬煮前体积 (pre_boil_volume) - 用于实际损失计算
pre_boil_volume = target_post_boil_volume + boil_off_volume

# 计算总水量（用于洗糟后提取系数计算和比重估计）
total_water_volume = strike_water + sparge_water

# 计算洗糟后可溶糖（使用洗糟后提取系数）
# M_act_post = post_sparge_coefficient × M_potential
sugar_post_sparge = potential_extract_total * post_sparge_coefficient

# 用总水量和糖量计算洗糟后比重（用于过程监控）
# 建立方程: sugar = 2.5 × (SG - 1) × total_water_volume × SG
a = 2.5 * total_water_volume
b = -2.5 * total_water_volume
c = -sugar_post_sparge
discriminant = b**2 - 4*a*c
estimated_pre_boil_sg_for_monitoring = (-b + discriminant**0.5) / (2*a)

# 验证：根据洗糟后比重计算煮沸后比重
sugar_post_boil = potential_extract_total * post_boil_efficiency
estimated_post_boil_sg = estimate_sg_from_sugar(target_post_boil_volume, sugar_post_boil)

print("📋 步骤5: 估计洗糟后比重并验证煮沸后比重")
print("-"*70)
print(f"\n【已知数据】")
print(f"  洗糟前提取系数 = {pre_sparge_coefficient:.4f} ({pre_sparge_coefficient*100:.2f}%)")
print(f"  洗糟后提取系数 = {post_sparge_coefficient:.4f} ({post_sparge_coefficient*100:.2f}%)")
print(f"  煮沸后效率 = {post_boil_efficiency:.4f} ({post_boil_efficiency*100:.2f}%)")
print(f"  麦芽总重量 = {total_malt_weight:.3f} kg，理论总糖量 = {potential_extract_total:.3f} kg")
print(f"  熬煮前体积 = {pre_boil_volume:.2f} L")
print(f"  总水量 = {total_water_volume:.2f} L (糖化{strike_water:.2f}L + 洗糟{sparge_water:.2f}L)")

print(f"\n【计算洗糟后比重】")
print(f"  ① 理论总糖量: {total_malt_weight:.3f} × {potential_extract_rate:.2f} = {potential_extract_total:.5f} kg")
print(f"  ② 洗糟后可溶糖: {potential_extract_total:.5f} × {post_sparge_coefficient:.4f} = {sugar_post_sparge:.5f} kg")
print(f"  ③ 建立方程: {sugar_post_sparge:.3f} = 2.5 × (SG - 1) × {total_water_volume:.2f} × SG")
print(f"  ✅ 洗糟后比重 SG = {estimated_pre_boil_sg_for_monitoring:.4f}")
print(f"     (基于总水量 {total_water_volume:.2f}L，用于过程监控)")

print(f"\n【验证煮沸后比重】")
print(f"  使用煮沸后效率：{post_boil_efficiency:.4f} ({post_boil_efficiency*100:.2f}%)")
print(f"  煮沸后糖量：{sugar_post_boil:.3f} kg")
print(f"  ✅ 煮沸后比重 SG = {estimated_post_boil_sg:.4f} (目标 {target_post_boil_sg:.4f})")

print("\n" + "="*70)

📋 步骤5: 估计洗糟后比重并验证煮沸后比重
----------------------------------------------------------------------

【已知数据】
  洗糟前提取系数 = 0.7077 (70.77%)
  洗糟后提取系数 = 0.7947 (79.47%)
  煮沸后效率 = 0.6869 (68.69%)
  麦芽总重量 = 6.528 kg，理论总糖量 = 5.092 kg
  熬煮前体积 = 23.70 L
  总水量 = 29.32 L (糖化20.89L + 洗糟8.43L)

【计算洗糟后比重】
  ① 理论总糖量: 6.528 × 0.78 = 5.09207 kg
  ② 洗糟后可溶糖: 5.09207 × 0.7947 = 4.04657 kg
  ③ 建立方程: 4.047 = 2.5 × (SG - 1) × 29.32 × SG
  ✅ 洗糟后比重 SG = 1.0524
     (基于总水量 29.32L，用于过程监控)

【验证煮沸后比重】
  使用煮沸后效率：0.6869 (68.69%)
  煮沸后糖量：3.498 kg
  ✅ 煮沸后比重 SG = 1.0600 (目标 1.0600)



## 5. 配方汇总

## 糖量平衡分析

In [48]:
# 计算总用水量
total_water = strike_water + sparge_water

# 创建配方汇总表格
recipe_summary = pd.DataFrame({
    '项目': [
        '========= 目标参数 ==========',
        '煮沸后体积 (L)',
        '煮沸后比重',
        '',
        '========= 原料用量 ==========',
        '麦芽总重量 (kg)',
        '理论浸出率 (%)',
        '',
        '========= 用水参数 ==========',
        '糖化用水量 (L)',
        '洗糟水用量 (L)',
        '总用水量 (L)',
        '水料比',
        '',
        '========= 过程监控 ==========',
        '糖化后比重估计值',
        '熬煮前体积 (L)',
        '洗糟后比重监控目标值',
        '煮沸后比重目标值',
        '熬煮蒸发量 (L)',
        ''
    ],
    '数值': [
        '',
        f'{target_post_boil_volume:.2f}',
        f'{target_post_boil_sg:.4f}',
        '',
        '',
        f'{total_malt_weight:.3f}',
        f'{potential_extract_rate*100:.0f}',
        '',
        '',
        f'{strike_water:.2f}',
        f'{sparge_water:.2f}',
        f'{total_water:.2f}',
        f'{water_grain_ratio:.1f}',
        '',
        '',
        f'{estimated_pre_sparge_sg:.4f}',
        f'{pre_boil_volume:.2f}',
        f'{estimated_pre_boil_sg_for_monitoring:.4f}',
        f'{target_post_boil_sg:.4f}',
        f'{boil_off_volume:.2f}',
        ''
    ]
})

print("\n" + "="*60)
print("📊 配方设计完成")
print("="*60)
display(recipe_summary)
print()
print("💡 监控说明：")
print(f"  - 糖化后比重目标: {estimated_pre_sparge_sg:.4f} (基于糖化水{strike_water:.2f}L)")
print(f"  - 洗糟后比重目标: {estimated_pre_boil_sg_for_monitoring:.4f} (基于总水量{total_water:.2f}L)")
print(f"  - 煮沸后比重目标: {target_post_boil_sg:.4f} (基于煮沸后体积{target_post_boil_volume:.2f}L)")
print("="*60)


📊 配方设计完成


,项目,数值
0,========= 目标参数 ==========,
1,煮沸后体积 (L),22.00
2,煮沸后比重,1.0600
3,,
4,========= 原料用量 ==========,
5,麦芽总重量 (kg),6.528
6,理论浸出率 (%),78
7,,
8,========= 用水参数 ==========,
9,糖化用水量 (L),20.89



💡 监控说明：
  - 糖化后比重目标: 1.0648 (基于糖化水20.89L)
  - 洗糟后比重目标: 1.0524 (基于总水量29.32L)
  - 煮沸后比重目标: 1.0600 (基于煮沸后体积22.00L)


In [49]:
# 糖量平衡分析
# 计算各阶段的实际糖量
potential_sugar_total = total_malt_weight * potential_extract_rate  # 理论总糖量
sugar_mash = potential_sugar_total * pre_sparge_coefficient  # 糖化后提取糖量
sugar_sparge = potential_sugar_total * post_sparge_coefficient  # 洗糟后糖量
sugar_boil = calculate_actual_extract(target_post_boil_volume, target_post_boil_sg)  # 煮沸后收获糖量

sugar_balance = pd.DataFrame({
    '阶段': [
        '1. 理论总糖量',
        '2. 糖化后提取',
        '3. 洗糟后收集',
        '4. 煮沸后收获'
    ],
    '糖量 (kg)': [
        f'{potential_sugar_total:.3f}',
        f'{sugar_mash:.3f}',
        f'{sugar_sparge:.3f}',
        f'{sugar_boil:.3f}'
    ],
    '累计提取系数': [
        '1.000',
        f'{pre_sparge_coefficient:.4f}',
        f'{post_sparge_coefficient:.4f}',
        f'{post_boil_efficiency:.4f}'
    ],
    '体积 (L)': [
        '-',
        f'{strike_water:.2f}',
        f'{total_water_volume:.2f}',
        f'{target_post_boil_volume:.2f}'
    ],
    '比重': [
        '-',
        f'{estimated_pre_sparge_sg:.4f}',
        f'{estimated_pre_boil_sg_for_monitoring:.4f}',
        f'{target_post_boil_sg:.4f}'
    ]
})

print("\n" + "="*90)
print("📊 糖量平衡与浓度变化分析")
print("="*90)
display(sugar_balance.style.set_properties(**{'text-align': 'center'}))
print()
print(f"糖量变化: {potential_sugar_total:.3f} → {sugar_mash:.3f} → {sugar_sparge:.3f} → {sugar_boil:.3f} kg")
print(f"体积变化: {strike_water:.2f} → {total_water_volume:.2f} → {target_post_boil_volume:.2f} L")
print(f"比重变化: {estimated_pre_sparge_sg:.4f} → {estimated_pre_boil_sg_for_monitoring:.4f} → {target_post_boil_sg:.4f}")
print("="*90)


📊 糖量平衡与浓度变化分析


,阶段,糖量 (kg),累计提取系数,体积 (L),比重
0,1. 理论总糖量,5.092,1.000,-,-
1,2. 糖化后提取,3.604,0.7077,20.89,1.0648
2,3. 洗糟后收集,4.047,0.7947,29.32,1.0524
3,4. 煮沸后收获,3.498,0.6869,22.00,1.0600



糖量变化: 5.092 → 3.604 → 4.047 → 3.498 kg
体积变化: 20.89 → 29.32 → 22.00 L
比重变化: 1.0648 → 1.0524 → 1.0600
